In [1]:
import pyspark.sql.functions as F

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

data = [
    [100, 1, 23, 10],
    [100, 2, 45, 11],
    [100, 3, 67, 12],
    [100, 4, 78, 13],
    [101, 1, 23, 10],
    [101, 2, 45, 13],
    [101, 3, 67, 14],
    [101, 4, 78, 15],
    [102, 1, 23, 10],
    [102, 2, 45, 11],
    [102, 3, 67, 16],
    [102, 4, 78, 18]
]

df = spark.createDataFrame(data, ['id', 'day', 'price', 'units'])

price_pivot = df.groupBy('id').pivot('day').agg(F.sum('price'))
price_pivot = price_pivot.select('id',
                                 *[F.col(col).alias(f'price_{col}') for col in price_pivot.columns if col != 'id'])

units_pivot = df.groupBy('id').pivot('day').agg(F.sum('units'))
units_pivot = units_pivot.select('id',
                                 *[F.col(col).alias(f'units_{col}') for col in units_pivot.columns if col != 'id'])

joined = price_pivot.join(units_pivot, 'id')
joined.show()

+---+-------+-------+-------+-------+-------+-------+-------+-------+
| id|price_1|price_2|price_3|price_4|units_1|units_2|units_3|units_4|
+---+-------+-------+-------+-------+-------+-------+-------+-------+
|100|     23|     45|     67|     78|     10|     11|     12|     13|
|101|     23|     45|     67|     78|     10|     13|     14|     15|
|102|     23|     45|     67|     78|     10|     11|     16|     18|
+---+-------+-------+-------+-------+-------+-------+-------+-------+

